# Load and format data into tables
This code finds data from previous runs of code and formats it nicely.

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# Code choices to decide what to make into tables.
version = "version23"
# Add the subfolders here for analysis, e.g. OPEC_Conference, OPEC_conf_29. The empty string generates the COP 
subfolderlist = [""]
# Name of folders to open
subsubfolder_labels = {
    "constsust_20": "Neutral rated 20", 
    "constsust_32": "Neutral rated 32", 
    "greencred_20": "Green rated 20",
    "greencred_32": "Green rated 32",
    "fossil_20": "Fossil 20", 
    "fossil_32": "Fossil 32", 
    "renewable_20": "Renewable 20",
    "renewable_all": "Renewable 32",
    #"SandPESG_32": "S&P ESG environmental 32",
    #"SandP_32": "S&P ESG overall 32",
}
# Directory of results. 
results_dir = "./output/"
# May want to use results with different length factors for ranking methods and OLS methods. 
afterlength = 0
olsafterlength = 0

# The neutral group whose behaviour is accounted for in the difference model, e.g. constsust_32 or SandP
neutralGroup = "constsust_32"
# Include the whole period results?
includeWholePeriod = False
# Include the alternative model of least squares fitting
includeSM = True
# The variable used to denote the time-value of money, either "Norm^IRX" or the empty string if none is used.
interestratename = "" # "_Norm^IRX"

# File names shouldn't need editing
delaystring = "" if ((afterlength==0)&(olsafterlength==0)) else f"Delay_{afterlength}_{olsafterlength}"
copnamestring = "_COPyear"
kstestfolder = f"before0_after{afterlength}_norm{neutralGroup}_1"
olstestfolder = f"before0_after{olsafterlength}_norm{neutralGroup}_cleaned"
kstestfilename = "Kolmogorov_Smirnoff_test_results.csv"
olsresultsfile = f"OLSsummary{interestratename}.txt"
olsresultsSqfile = f"OLSsummaryDailyVar{interestratename}.txt"
olsresultsCopYearfile = f"OLSsummary_COPyear{interestratename}.txt"
olsresultsSqCopYearfile = f"OLSsummaryDailyVar_COPyear{interestratename}.txt"
FEOLS_filename_diff = f"FEOLSdataSummaryDayChange_x{copnamestring}{interestratename}.csv"
FEOLS_filename_range = f"FEOLSdataSummaryDayVar_x{copnamestring}{interestratename}.csv"

diffCOPYearTrendFile = "COPyearDiffTrend.csv"
varCOPYearTrendFile = "COPyearVarianceTrend.csv"

In [3]:
# Define some functions required to read the COP values from the OLS regression table
def extract_cop_value(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line.strip().startswith("COP"):
            # Split the line to find the COP value
            parts = line.replace(",", " ").split()
            if len(parts) > 1:
                # Reformat the percentage for convenience
                parts = [float(x) for x in parts[1:]]
                return parts
    return None  # Return None if "COP" is not found in the file

# This function reads the COP-year values from the OLS regression tables
def extract_cop_year_value(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line.strip().startswith("COPnumbers"):
            # Split the line to find the COP value
            parts = line.replace(",", " ").split()
            if len(parts) > 1:
                # Reformat the percentage for convenience
                parts = [float(x) for x in parts[1:]]
                return parts
    return None  # Return None if "COP" is not found in the file

# Function to read the trend line experiment data
def extract_cop_year_trend(file_path):
    df = pd.read_csv(file_path, index_col=0)
    return df.loc["x1", :]

In [4]:
subsubfolder_labels

{'constsust_20': 'Neutral rated 20',
 'constsust_32': 'Neutral rated 32',
 'greencred_20': 'Green rated 20',
 'greencred_32': 'Green rated 32',
 'fossil_20': 'Fossil 20',
 'fossil_32': 'Fossil 32',
 'renewable_20': 'Renewable 20',
 'renewable_all': 'Renewable 32'}

In [5]:
def convert_one_tailed_to_two(p):
    return 1-2 * abs(p-0.5)

In [6]:
prec = 4  # Precision in table (# decimal places)
for subfolder in subfolderlist:
    results = []
    results_sup = []
    for key, val in subsubfolder_labels.items():
        tempolstestfolder = olstestfolder.replace(neutralGroup, "None") if ("constsust" in key) else olstestfolder
        if includeWholePeriod:
            tempkstestfolder = kstestfolder.replace(neutralGroup, "None") if ("constsust" in key) else kstestfolder
            subresult = pd.read_csv(
                os.path.join(results_dir, version, subfolder, key, tempkstestfolder, kstestfilename)
            )
            if type(subresult["testStat"][0]) == type(""):
                subresult = subresult.iloc[:-1, :]
                subresult["testStat"] = pd.to_numeric(subresult["testStat"])
            diffPeriodCOPyearTrend = extract_cop_year_trend(
                 os.path.join(results_dir, version, subfolder, key, tempkstestfolder, diffCOPYearTrendFile)
            )
            varPeriodCOPyearTrend = extract_cop_year_trend(
                 os.path.join(results_dir, version, subfolder, key, tempkstestfolder, varCOPYearTrendFile)
            )
        subOLSresult = extract_cop_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, olsresultsfile)
        )
        subOLSsqresult = extract_cop_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, olsresultsSqfile)
        )
        subOLSresultCOPyearTerm = extract_cop_year_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, olsresultsCopYearfile)
        )
        subOLSsqresultCOPyearTerm = extract_cop_year_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, olsresultsSqCopYearfile)
        )
        FEOLS_file_diff_use = FEOLS_filename_diff.replace("_x", "") if ("constsust" in key) else FEOLS_filename_diff
        FEOLS_file_range_use = FEOLS_filename_range.replace("_x", "") if ("constsust" in key) else FEOLS_filename_range
        FEOLS_COP_diff = extract_cop_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, FEOLS_file_diff_use.replace(copnamestring, ""))
        )
        FEOLS_COP_range = extract_cop_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, FEOLS_file_range_use.replace(copnamestring, ""))
        )
        FEOLS_COP_trend_diff = extract_cop_year_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, FEOLS_file_diff_use)
        )
        FEOLS_COP_trend_range = extract_cop_year_value(
            os.path.join(results_dir, version, subfolder, key, tempolstestfolder, FEOLS_file_range_use)
        )
        
        if includeWholePeriod:
            results.append(
                pd.Series({
                    "Portfolio": val,
                    "Difference rank": f'{subresult.loc[subresult["Test"]=="RankTestDiff", "testStat"].iloc[0]:.3f} ({convert_one_tailed_to_two(subresult.loc[subresult["Test"]=="RankTestDiff", "pval"].iloc[0]):.3f})', 
                    "gSD rank": f'{subresult.loc[subresult["Test"]=="RankTestgeoSD", "testStat"].iloc[0]:.3f} ({convert_one_tailed_to_two(subresult.loc[subresult["Test"]=="RankTestgeoSD", "pval"].iloc[0]):.3f})', 
                    "COP linear diff pyfixedefs": f'{FEOLS_COP_diff[1]:.{prec}f} ({FEOLS_COP_diff[3]:.{prec}f})' + ("*" if (FEOLS_COP_diff[3] < 0.05) else ""),
                    "COP linear range pyfixedefs": f'{FEOLS_COP_range[1]:.{prec}f} ({FEOLS_COP_range[3]:.{prec}f})' + ("*" if (FEOLS_COP_range[3] < 0.05) else "")

                })
            )
            results_sup.append(
                pd.Series({
                    "Portfolio": val,
                    "COP period trend diff fit": f'{diffPeriodCOPyearTrend["BestEstimate"]:.{prec}f} [{diffPeriodCOPyearTrend["LowerConf"]:.{prec}f} - {diffPeriodCOPyearTrend["UpperConf"]:.4f}]',
                    "COP period trend gSD fit": f'{diffPeriodCOPyearTrend["BestEstimate"]:.{prec}f} [{diffPeriodCOPyearTrend["LowerConf"]:.{prec}f} - {diffPeriodCOPyearTrend["UpperConf"]:.4f}]',
                    "COP diff trend pyfixedefs": f'{FEOLS_COP_trend_diff[1]:.{prec}f} ({FEOLS_COP_trend_diff[3]:.{prec}f})' + ("*" if (FEOLS_COP_trend_diff[3]<0.05) else ""),
                    "COP range trend pyfixedefs": f'{FEOLS_COP_trend_range[1]:.{prec}f} ({FEOLS_COP_trend_range[3]:.{prec}f})' + ("*" if (FEOLS_COP_trend_range[3]<0.05) else "")

                })
            )
        else:
            results.append(
                pd.Series({
                    "Portfolio": val,
                    "COP linear diff pyfixedefs": f'{FEOLS_COP_diff[1]:.{prec}f} ({FEOLS_COP_diff[3]:.{prec}f})' + ("*" if (FEOLS_COP_diff[3] < 0.05) else ""),
                    "COP linear range pyfixedefs": f'{FEOLS_COP_range[1]:.{prec}f} ({FEOLS_COP_range[3]:.{prec}f})' + ("*" if (FEOLS_COP_range[3] < 0.05) else "")
                })
            )
            results_sup.append(
                pd.Series({
                    "Portfolio": val,
                    "COP trend linear diff pyfixedefs": f'{subOLSresultCOPyearTerm[0]:.{prec}f} ({subOLSresultCOPyearTerm[3]:.{prec}f})' + ("*" if (subOLSresultCOPyearTerm[3] < 0.05) else ""),
                    "COP trend linear range pyfixedefs": f'{subOLSsqresultCOPyearTerm[0]:.{prec}f} ({subOLSsqresultCOPyearTerm[3]:.{prec}f})' + ("*" if (subOLSsqresultCOPyearTerm[3] < 0.05) else ""),

                })
            )
        if includeSM:
            results[-1] = pd.concat([
                    results[-1], 
                    pd.Series({
                        "COP linear diff term": f'{subOLSresult[0]:.{prec}f} ({subOLSresult[3]:.{prec}f})' + ("*" if (subOLSresult[3] < 0.05) else ""),
                        "COP linear range term": f'{subOLSsqresult[0]:.{prec}f} ({subOLSsqresult[3]:.{prec}f})' + ("*" if (subOLSsqresult[3] < 0.05) else ""),
                    }),
                ])
            results_sup[-1] = pd.concat([
                results_sup[-1], 
                pd.Series({
                    "COP trend linear diff term": f'{subOLSresultCOPyearTerm[0]:.{prec}f} ({subOLSresultCOPyearTerm[3]:.{prec}f})' + ("*" if (subOLSresultCOPyearTerm[3] < 0.05) else ""),
                    "COP trend linear range term": f'{subOLSsqresultCOPyearTerm[0]:.{prec}f} ({subOLSsqresultCOPyearTerm[3]:.{prec}f})' + ("*" if (subOLSsqresultCOPyearTerm[3] < 0.05) else ""),
                })  
            ])
    results = pd.DataFrame(results)
    results.to_csv(os.path.join(results_dir, version, subfolder, f"ResultsSummaryTable{interestratename}_Norm{neutralGroup}{delaystring}.csv"))
    results_sup = pd.DataFrame([r.to_dict() for r in results_sup])
    results_sup.to_csv(
        os.path.join(results_dir, version, subfolder, f"ResultsSummarySupplementaryTable{interestratename}_Norm{neutralGroup}{delaystring}.csv")
    )

In [7]:
results

,Portfolio,COP linear diff pyfixedefs,COP linear range pyfixedefs,COP linear diff term,COP linear range term
0,Neutral rated 20,0.0002 (0.3992),0.0003 (0.0825),0.0002 (0.4591),0.0005 (0.0013)*
1,Neutral rated 32,0.0002 (0.3975),0.0002 (0.0430)*,0.0002 (0.4512),0.0005 (0.0003)*
2,Green rated 20,0.0003 (0.0001)*,0.0003 (0.3413),0.0013 (0.0001)*,0.0003 (0.1527)
3,Green rated 32,0.0002 (0.0002)*,0.0002 (0.1896),0.0009 (0.0002)*,0.0003 (0.0542)
4,Fossil 20,0.0002 (0.0501),0.0003 (0.9521),-0.0005 (0.0673),0.0000 (0.9301)
5,Fossil 32,0.0002 (0.0677),0.0003 (0.8779),-0.0004 (0.0744),0.0001 (0.3628)
6,Renewable 20,0.0005 (0.1726),0.0006 (0.1182),-0.0007 (0.2283),0.0009 (0.0145)*
7,Renewable 32,0.0004 (0.1535),0.0006 (0.1083),-0.0006 (0.2001),0.0009 (0.0113)*


In [8]:
results_sup

,Portfolio,COP trend linear diff pyfixedefs,COP trend linear range pyfixedefs,COP trend linear diff term,COP trend linear range term
0,Neutral rated 20,-0.0000 (0.9613),0.0000 (0.5306),-0.0000 (0.9613),0.0000 (0.5306)
1,Neutral rated 32,0.0000 (0.7980),-0.0000 (0.9263),0.0000 (0.7980),-0.0000 (0.9263)
2,Green rated 20,0.0000 (0.7864),-0.0000 (0.1797),0.0000 (0.7864),-0.0000 (0.1797)
3,Green rated 32,0.0001 (0.0662),-0.0000 (0.1572),0.0001 (0.0662),-0.0000 (0.1572)
4,Fossil 20,-0.0001 (0.0045)*,-0.0001 (0.0002)*,-0.0001 (0.0045)*,-0.0001 (0.0002)*
5,Fossil 32,-0.0001 (0.0032)*,-0.0000 (0.0316)*,-0.0001 (0.0032)*,-0.0000 (0.0316)*
6,Renewable 20,-0.0002 (0.0102)*,0.0001 (0.0706),-0.0002 (0.0102)*,0.0001 (0.0706)
7,Renewable 32,-0.0001 (0.0574),0.0002 (0.0000)*,-0.0001 (0.0574),0.0002 (0.0000)*


# Plot ESG comparisons

In [11]:
ratings = pd.read_csv("./input/companiesmarketcap.com - Companies ranked by Market Cap - CompaniesMarketCap.com.csv")

In [14]:
ratings.corr(method="pearson")

,Rank,marketcap,price (USD),Sustainalytics value,Sustainalytics 20-30,Sustainalytics <20,Sustainalytics >30,Unnamed: 11,S&P ESG environmental,S&P ESG,Unnamed: 14,Unnamed: 15,Unnamed: 16,Rank sustainalytics,Rank S&P,Unnamed: 19
Rank,1.000000,-0.251557,-0.132013,-0.112814,-0.346905,0.433565,-0.072949,NaN,0.039852,0.190374,NaN,NaN,NaN,0.108612,-0.017011,-0.171475
marketcap,-0.251557,1.000000,0.061896,0.100870,0.079267,-0.111027,-0.009509,NaN,-0.037399,-0.154949,NaN,NaN,NaN,-0.196566,0.138675,0.231930
price (USD),-0.132013,0.061896,1.000000,-0.227237,-0.092921,0.190091,-0.131990,NaN,0.005933,-0.001764,NaN,NaN,NaN,0.448421,-0.257168,-0.235488
Sustainalytics value,-0.112814,0.100870,-0.227237,1.000000,0.261623,-0.654345,0.644923,NaN,-0.505020,-0.497042,NaN,NaN,NaN,-0.929072,0.641157,0.598769
Sustainalytics 20-30,-0.346905,0.079267,-0.092921,0.261623,1.000000,-0.720243,-0.224934,NaN,-0.016485,-0.050836,NaN,NaN,NaN,-0.336320,-0.079131,0.053483
Sustainalytics <20,0.433565,-0.111027,0.190091,-0.654345,-0.720243,1.000000,-0.300668,NaN,0.318057,0.320283,NaN,NaN,NaN,0.875814,-0.449071,-0.498877
Sustainalytics >30,-0.072949,-0.009509,-0.131990,0.644923,-0.224934,-0.300668,1.000000,NaN,-0.318176,-0.307435,NaN,NaN,NaN,-0.609290,0.478951,0.399100
Unnamed: 11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S&P ESG environmental,0.039852,-0.037399,0.005933,-0.505020,-0.016485,0.318057,-0.318176,NaN,1.000000,0.873761,NaN,NaN,NaN,0.619692,-0.974435,-0.841170
S&P ESG,0.190374,-0.154949,-0.001764,-0.497042,-0.050836,0.320283,-0.307435,NaN,0.873761,1.000000,NaN,NaN,NaN,0.625328,-0.860466,-0.983339


In [ ]:
ratings